<a href="https://colab.research.google.com/github/snipaid-nlg/models/blob/main/news-headline-generation/news-headline-generation-with-flan-t5-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Headline Generation with flan-t5-base
Evaluating flan-t5-base on the task of generating headlines from journalistic text in german.

## Installs and Imports

In [ ]:
!pip install -q transformers datasets accelerate sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.8/212.8 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 614.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset, Dataset
from transformers.pipelines.pt_utils import KeyDataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, pipeline
import pandas as pd
from tqdm.auto import tqdm

## Loading and transforming 10kGNAD
We load the [Ten Thousand German News Articles Dataset](https://tblock.github.io/10kGNAD/) from Huggingface. The first sentence of a text (up to the first period) is a headline, so we split the text and use the second part. A Prefix `summarize:` is added because the model was trained on the task of summarization.

In [ ]:
gnad10 = load_dataset("gnad10")
df = pd.DataFrame(gnad10["train"])
df[['headline', 'fulltext']] = df["text"].str.split('.', n=1, expand=True)
df.drop(['label','text'], axis=1, inplace=True)
df['fulltext'] = "summarize: " + df['fulltext']
df.head()

Generating train split:   0%|          | 0/9245 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1028 [00:00<?, ? examples/s]

Dataset gnad10 downloaded and prepared to /root/.cache/huggingface/datasets/gnad10/default/1.1.0/3a8445be65795ad88270af4d797034c3d99f70f8352ca658c586faf1cf960881. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

,headline,fulltext
0,21-Jähriger fällt wohl bis Saisonende aus,summarize: Wien – Rapid muss wohl bis Saisone...
1,"Erfundene Bilder zu Filmen, die als verloren g...",summarize: Wien – Die Filmgeschichte ist ein ...
2,Der frischgekürte CEO Sundar Pichai setzt auf ...,summarize: Die Atmosphäre im Silicon Valley i...
3,"Putin: ""Einigung, dass wir Menge auf Niveau vo...",summarize: Moskau – Die russischen Ölproduzen...
4,Estland sieht den künftigen österreichischen P...,summarize: Wien/Tallinn/Pskow – Die Eltern de...


Converting the pandas dataframe to a Huggingface dataset object

In [ ]:
dataset = Dataset.from_pandas(df)

## Loading the model

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", use_cache=True, device_map='auto')
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base", padding=True)

In [ ]:
template = {
    "num_beams": 1,
    "max_new_tokens": None,
    "max_length" : 512,
    "length_penalty": 1.0,
    "min_new_tokens": 5,
    "no_repeat_ngram_size": 0,
    "repetition_penalty": 1.0,
    "diversity_penalty": 0.0,
    "num_beam_groups": 1,
    "do_sample": False,
    "top_k": 50,
    "top_p": 1.0,
    "temperature": 1.0,
    "early_stopping": True,
    "pad_token_id": 0,
    "eos_token_id": 1,
    "num_return_sequences": 1
}

In [ ]:
pipe = pipeline("text2text-generation",
                device=0,
                model=model, 
                tokenizer=tokenizer, 
                **template)

In [ ]:
for out in tqdm(pipe(KeyDataset(dataset, "fulltext"))):
    print(out)

  0%|          | 0/9245 [00:00<?, ?it/s]

[{'generated_text': '– Rapid must be a contender to the title, but he must be able to keep his place in the top ten.'}]
[{'generated_text': 'Wien – The film history is a Friedhof of the Verlorenen. Under the Begrabenen find numble films, from which only still mysterios or abenteuerly klingende names kursieren; and such, over the transgressed herumliegende secondary material output allows. One of them is The Forbidden Room, a two-reeler from 1913/14, den der arbeitswütige US-Regisseur Allan Dwan u. a. with the horrordarsteller Lon Chaney gedreht u. a. with [...]'}]
[{'generated_text': 'In Silicon Valley, massenhaft Mitarbeitern der direkten Konkurrenz stören, Löhne mit firmenübergreifenden Mauscheleien niedrig gehalten und unterstreichend durch leicht verfehlter Ziele vor der Tür.'}]
[{'generated_text': 'Moscow – Russian ol producers want to be supported this year, according to President Wladimir Putin’s words.'}]


KeyboardInterrupt: ignored